In [ ]:
!pip install -q chromadb sentence-transformers --quiet

In [4]:
import chromadb
from chromadb.config import Settings
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
from datetime import datetime
import os

persist_dir = "./chroma_metadata_memory"
if not os.path.exists(persist_dir):
  os.makedirs(persist_dir)


In [6]:
embedding_fn = SentenceTransformerEmbeddingFunction(model_name = "all-MiniLM-L6-v2")
client= chromadb.PersistentClient(path = persist_dir)



In [7]:
collection = client.get_or_create_collection(name="topic_memory",embedding_function=embedding_fn)


In [8]:
def store_message(topic,message,role="user"):
  timestamp = datetime.now().isoformat()
  uid= f"{role}_{timestamp}"
  collection.add(documents=[message], metadatas=[{"topic":topic, "timestamp": timestamp, "role": role }], ids=[uid])

In [13]:
def get_messages_by_topic(topic):
  results = collection.query(
      query_texts=["dummy"],
      n_results=10,
      where={"topic": topic}
  )
  return results.get("documents",[[]])[0]

In [14]:
def clear_topic(topic):
  collection.delete(where={"topic": topic})

In [17]:
print("Topic Memory Bot (use: 'exit', 'show topic', 'clear topic')\n")
while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        break
    elif user_input.lower().startswith("show "):
        topic = user_input.split(" ", 1)[1]
        history = get_messages_by_topic(topic)
        if history:
            print(f"\nMessages for topic '{topic}':")
            for msg in history:
                print(" -", msg)
        else:
            print(f"No records found for topic '{topic}'.")
        continue
    elif user_input.lower().startswith("clear "):
        topic = user_input.split(" ", 1)[1]
        clear_topic(topic)
        print(f"Cleared memory for topic '{topic}'.")
        continue
    elif ":" in user_input:
        topic, message = user_input.split(":", 1)
        store_message(topic.strip(), message.strip())
        print(f"Agent: Noted under topic '{topic.strip()}'.")
    else:
        print("Please enter in format '<topic>: <message>'")

Topic Memory Bot (use: 'exit', 'show topic', 'clear topic')

You: show DS
No records found for topic 'DS'.
You: show python

Messages for topic 'python':
 - Starting
You: exit
